<a href="https://colab.research.google.com/github/hamish-haggerty/AI-hacking/blob/master/SSL/cancer_validation_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cancer_validation_ensemble

> Purpose of this notebook is simply to run exploratory experiments to compare ensembling: 2 pretrained models under 3 types of initial weights: (BT,BT),(Sup,Sup),(BT,Sup)

In [1]:
#| default_exp cancer_validation_ensemble

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [2]:
#| hide

import os
from google.colab import drive

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():

    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')

    os.chdir('cancer-proj')
    
    os.system('pip install .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

Mounted at /content/drive


In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *
from cancer_proj.cancer_maintrain import *

## Load the data

In [5]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [6]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

## Optionally, display:

In [7]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [8]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [9]:
#| export

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

Need to run a few exploratory experiments. Based on the results, next is to run some systematic experiments, probably with W and B... Or final results...

## Experiment: Check how initial_weights=BT_pretrain goes, when using new default fine_tuning:

In [10]:
#| export

@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tuner to use with bt initial weights"
    
    self.freeze() #freeze the resnet
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    #self.unfreeze() #don't unfreeze the resnet. We are fitting training the encoder head + projector
    #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

    self.unfreeze() #We can unfreeze at the end

In [11]:
#| export

class NLM(nn.Module):
    "Basic nonlinear "
    def __init__(self,encoder,device='cuda'):
        super().__init__()

        self.encoder=encoder
        # self.fc1 = nn.Linear(2048,2048)
        # self.bn1 = nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        # self.relu = nn.ReLU(inplace=True)
        # self.head=nn.Linear(2048,9)

        self.head = sequential(nn.Linear(2048,2048),nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                               nn.ReLU(inplace=True),nn.Linear(2048,9))
        
        self.device = torch.device(device)
        self.to(self.device)


    def forward(self,x):
        x=self.encoder(x)
        x=self.head(x)

        # x=self.encoder(x)
        # x=self.fc1(x)
        # x=self.bn1(x)
        # x=self.relu(x)
        # x=self.head(x)
        return x

# First we verify that adding non linear head still works. This should only take 1 run: 

## Test: make sure that it freezes the appropriate part of model:

In [12]:
# bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
# model = NLM(encoder)
# test_eq(len(my_splitter(model)),2)
# test_eq(len(my_splitter_bt(bt_model)),2)

# learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
# learn.freeze()
# print('body should be frozen, (sans batchnorm) head unfrozen')
# learn.summary()

## Check that adding head works ok

In [13]:

# initial_weights = 'supervised_pretrain'
# pretrain=False
# numfit=50
# num_epochs=10
# #main.freeze_encoder=False

# main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
#         aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
#         initial_weights=initial_weights,pretrain=pretrain,num_epochs=num_epochs,numfit=numfit,
#         print_report=True,
#                  )


# metrics = main()



## Based on experimentation below seems we might need to train BT head for longer?

In [14]:
#Non default inputs
initial_weights = 'bt_pretrain'
pretrain=False
numfit=50
num_epochs='na'
freeze_num_epochs = 'na'
freeze_numfit=6

main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
        aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
        initial_weights=initial_weights,pretrain=pretrain,
        num_epochs=num_epochs,numfit=numfit,freeze_num_epochs=freeze_num_epochs,freeze_numfit=freeze_numfit,
        print_report=True,
                 )

metrics = main()

Downloading: "https://github.com/facebookresearch/barlowtwins/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/barlowtwins/ep1000_bs2048_lrw0.2_lrb0.0048_lambd0.0051/resnet50.pth" to /root/.cache/torch/hub/checkpoints/resnet50.pth


  0%|          | 0.00/90.0M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,time
0,2.197503,None,00:13
1,2.196716,None,00:06
2,2.192781,None,00:06
3,2.186036,None,00:06
4,2.176020,None,00:06
5,2.162937,None,00:06


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,2.068448,None,00:07
1,2.063880,None,00:06
2,2.057353,None,00:06
3,2.050801,None,00:06
4,2.044308,None,00:06
5,2.033850,None,00:06
6,2.024127,None,00:06
7,2.011858,None,00:06
8,1.996863,None,00:06
9,1.979338,None,00:07


                            precision    recall  f1-score   support

         actinic keratosis       0.57      0.65      0.60        20
      basal cell carcinoma       0.80      0.80      0.80        20
            dermatofibroma       0.88      0.74      0.80        19
                  melanoma       0.57      0.60      0.59        20
                     nevus       0.67      0.70      0.68        20
pigmented benign keratosis       0.60      0.45      0.51        20
      seborrheic keratosis       0.50      0.40      0.44        15
   squamous cell carcinoma       0.54      0.70      0.61        20
           vascular lesion       0.90      0.90      0.90        20

                  accuracy                           0.67       174
                 macro avg       0.67      0.66      0.66       174
              weighted avg       0.67      0.67      0.67       174



## Ok, we train BT head for 6 epochs instead of 3 for supervised. Makes sense.

## Note: adding simple non-linear head got 0.6954 (supervised). Also we set freeze_numfit=3. So this is our baseline. 

## Ensemble experiment: Let's train 3 models, for each kind of initial weights. Then we can compare, pairwise: ensembling 'within' models (i.e. same kind of initial weights) vs. 'between' models (i.e. different kinds of initial weights).

In [16]:
def run_main_train(initial_weights,freeze_numfit,num=5):
    "run main_train num times."

    main_dict = {}
    for i in range(num):

        main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
                aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
                initial_weights=initial_weights,pretrain=pretrain,
                num_epochs=num_epochs,numfit=numfit,freeze_num_epochs=freeze_num_epochs,freeze_numfit=freeze_numfit,
                print_report=True,
                        )
        
        metrics = main()
        main_dict[i] = metrics

    return main_dict
        

In [17]:
initial_weights_list = ['supervised_pretrain','bt_pretrain']
freeze_numfit_dict = {'supervised_pretrain':3,'bt_pretrain':6}
numfit=50
pretrain=False
results={'supervised_pretrain':None,'bt_pretrain':None}  

for initial_weights in initial_weights_list:
    main_dict = run_main_train(initial_weights=initial_weights,freeze_numfit=freeze_numfit_dict[initial_weights],num=3)
    results[initial_weights] = main_dict #main_dict has result of running main num=3 times.


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,time
0,2.207973,None,00:06
1,2.200081,None,00:07
2,2.173095,None,00:06


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,2.016304,None,00:06
1,1.955085,None,00:06
2,1.878572,None,00:07
3,1.786057,None,00:06
4,1.700708,None,00:06
5,1.600370,None,00:07
6,1.492862,None,00:06
7,1.381611,None,00:06
8,1.273624,None,00:06
9,1.185682,None,00:06


...

Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


epoch,train_loss,valid_loss,time
0,2.196029,None,00:07
1,2.194222,None,00:06
2,2.190600,None,00:06
3,2.184578,None,00:07
4,2.174982,None,00:06
5,2.162496,None,00:06


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,2.071273,None,00:06
1,2.065238,None,00:06
2,2.055900,None,00:07
3,2.050991,None,00:06
4,2.041942,None,00:06
5,2.033857,None,00:06
6,2.023553,None,00:06
7,2.009935,None,00:06
8,1.994901,None,00:06
9,1.975314,None,00:06


                            precision    recall  f1-score   support

         actinic keratosis       0.52      0.65      0.58        20
      basal cell carcinoma       0.65      0.75      0.70        20
            dermatofibroma       0.81      0.68      0.74        19
                  melanoma       0.40      0.30      0.34        20
                     nevus       0.67      0.70      0.68        20
pigmented benign keratosis       0.64      0.45      0.53        20
      seborrheic keratosis       0.29      0.27      0.28        15
   squamous cell carcinoma       0.56      0.70      0.62        20
           vascular lesion       0.81      0.85      0.83        20

                  accuracy                           0.60       174
                 macro avg       0.59      0.59      0.59       174
              weighted avg       0.60      0.60      0.60       174



In [ ]:
def get_ensemble_acc

#Print out result of each run and save: 

In [18]:
for initial_weights in initial_weights_list:

    print(f"With initial_weights={initial_weights}, results are: {[results[initial_weights][i]['acc'] for i in range(len(results[initial_weights]))]}")


With initial_weights=supervised_pretrain, results are: [0.6494252681732178, 0.6379310488700867, 0.6436781883239746]
With initial_weights=bt_pretrain, results are: [0.5977011322975159, 0.6091954112052917, 0.6034482717514038]


In [19]:
save_dict_to_gdrive(results,'base_results')

## First observation: results are all very similar, within type of initial weights. Let's first ensemble within models:

In [30]:
list(supervised)

{0: {'actinic keratosis': {'precision': 0.5416666666666666,
   'recall': 0.65,
   'f1-score': 0.5909090909090908,
   'support': 20},
  'basal cell carcinoma': {'precision': 0.75,
   'recall': 0.75,
   'f1-score': 0.75,
   'support': 20},
  'dermatofibroma': {'precision': 0.8,
   'recall': 0.8421052631578947,
   'f1-score': 0.8205128205128205,
   'support': 19},
  'melanoma': {'precision': 0.5263157894736842,
   'recall': 0.5,
   'f1-score': 0.5128205128205129,
   'support': 20},
  'nevus': {'precision': 0.7142857142857143,
   'recall': 0.75,
   'f1-score': 0.7317073170731706,
   'support': 20},
  'pigmented benign keratosis': {'precision': 0.8333333333333334,
   'recall': 0.5,
   'f1-score': 0.625,
   'support': 20},
  'seborrheic keratosis': {'precision': 0.35294117647058826,
   'recall': 0.4,
   'f1-score': 0.37500000000000006,
   'support': 15},
  'squamous cell carcinoma': {'precision': 0.4782608695652174,
   'recall': 0.55,
   'f1-score': 0.5116279069767442,
   'support': 20},
  '

In [31]:
sup_results = list(results['supervised_pretrain'].values())
sup_results

[{'actinic keratosis': {'precision': 0.5416666666666666,
   'recall': 0.65,
   'f1-score': 0.5909090909090908,
   'support': 20},
  'basal cell carcinoma': {'precision': 0.75,
   'recall': 0.75,
   'f1-score': 0.75,
   'support': 20},
  'dermatofibroma': {'precision': 0.8,
   'recall': 0.8421052631578947,
   'f1-score': 0.8205128205128205,
   'support': 19},
  'melanoma': {'precision': 0.5263157894736842,
   'recall': 0.5,
   'f1-score': 0.5128205128205129,
   'support': 20},
  'nevus': {'precision': 0.7142857142857143,
   'recall': 0.75,
   'f1-score': 0.7317073170731706,
   'support': 20},
  'pigmented benign keratosis': {'precision': 0.8333333333333334,
   'recall': 0.5,
   'f1-score': 0.625,
   'support': 20},
  'seborrheic keratosis': {'precision': 0.35294117647058826,
   'recall': 0.4,
   'f1-score': 0.37500000000000006,
   'support': 15},
  'squamous cell carcinoma': {'precision': 0.4782608695652174,
   'recall': 0.55,
   'f1-score': 0.5116279069767442,
   'support': 20},
  'vas

In [35]:
from itertools import combinations

print('Results for ensembling within supervised weights:')

sup_results = list(results['supervised_pretrain'].values())
sup_results = list(combinations(sup_results,2)) #all pairs of results. So for num=3, will be 3
for v in sup_results:

    print(f"Acc of first guy in ensemble is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}')
    
    

Results for ensembling within supervised weights:

Acc of first guy in ensemble is: 0.6494252681732178
Acc of second guy in ensemble is: 0.6379310488700867
Acc of ensemble is:0.6609195470809937


Acc of first guy in ensemble is: 0.6494252681732178
Acc of second guy in ensemble is: 0.6436781883239746
Acc of ensemble is:0.6436781883239746


Acc of first guy in ensemble is: 0.6379310488700867
Acc of second guy in ensemble is: 0.6436781883239746
Acc of ensemble is:0.6436781883239746



Helped on one, not on two. But of course very low sample size (i.e. only 3)

In [37]:
print('Results for ensembling within bt weights:')

bt_results = list(results['bt_pretrain'].values())
bt_results = list(combinations(bt_results,2)) #all pairs of results. So for num=3, will be 3
for v in sup_results:

    print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}\n')


Results for ensembling within bt weights:

Acc of first guy in ensemble is: 0.5977011322975159
Acc of second guy in ensemble is: 0.6091954112052917
Acc of ensemble is:0.6321839094161987


Acc of first guy in ensemble is: 0.5977011322975159
Acc of second guy in ensemble is: 0.6034482717514038
Acc of ensemble is:0.6091954112052917


Acc of first guy in ensemble is: 0.6091954112052917
Acc of second guy in ensemble is: 0.6034482717514038
Acc of ensemble is:0.6206896305084229



So, helped on two, not on one; but of course very low sample size. 

Now let's look at ensembling BETWEEN models:

In [48]:
print('Results for ensembling between models')
import itertools
sup_bt_results = list(itertools.product(results['supervised_pretrain'].values(),results['bt_pretrain'].values()))

for v in sup_bt_results:

    print(f"\nAcc of first guy in ensemble (supervised) is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble (bt) is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}\n')


Results for ensembling between models

Acc of first guy in ensemble (supervised) is: 0.6494252681732178
Acc of second guy in ensemble (bt) is: 0.5977011322975159
Acc of ensemble is:0.6724137663841248


Acc of first guy in ensemble (supervised) is: 0.6494252681732178
Acc of second guy in ensemble (bt) is: 0.6091954112052917
Acc of ensemble is:0.6666666865348816


Acc of first guy in ensemble (supervised) is: 0.6494252681732178
Acc of second guy in ensemble (bt) is: 0.6034482717514038
Acc of ensemble is:0.6666666865348816


Acc of first guy in ensemble (supervised) is: 0.6379310488700867
Acc of second guy in ensemble (bt) is: 0.5977011322975159
Acc of ensemble is:0.6494252681732178


Acc of first guy in ensemble (supervised) is: 0.6379310488700867
Acc of second guy in ensemble (bt) is: 0.6091954112052917
Acc of ensemble is:0.6551724076271057


Acc of first guy in ensemble (supervised) is: 0.6379310488700867
Acc of second guy in ensemble (bt) is: 0.6034482717514038
Acc of ensemble is:0.64

In [ ]:
#| hide

# #old supervised baseline (with fine tune)

# tem = {0: 0.6724137663841248,
#  1: 0.7126436829566956,
#  2: 0.6724137663841248,
#  3: 0.6321839094161987,
#  4: 0.6896551847457886}

# from statistics import mean
# mean(list(tem.values()))